In [5]:
import cv2
import os
import numpy as np
import random

### Get the fences and image filepaths in two lists

In [6]:
fences = [f'Fences/{file}' for file in os.listdir('Fences/') if file != '.DS_Store']
images = [f'Images/{file}' for file in os.listdir('Images/') if file != '.DS_Store']

sample_image = cv2.imread(images[0], cv2.IMREAD_UNCHANGED)
image_height = sample_image[0]
image_width = sample_image[1]

IndexError: list index out of range

### Helper Functions
These functions retrieve a random fence from the list of fences and view a fence as a preview

In [7]:
def Get_Random_Fence():
    fence = random.choice(fences)
    return cv2.imread(fence, cv2.IMREAD_UNCHANGED)

# Press esc twice to close the img
def View_Image(img):
    cv2.startWindowThread()
    cv2.imshow("Fence", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey()

In [8]:
def Get_Random_Crop(fence):
    max_y = fence.shape[0] - image_height
    max_x = fence.shape[1] - image_width
    y = np.random.randint(0, max_y)
    x = np.random.randint(0, max_x)
    return fence[y: y + image_height, x: x + image_width]


Test these methods to get a fence and crop it

In [ ]:
fence = Get_Random_Fence()
View_Image(fence)
View_Image(Get_Random_Crop(fence))

## Potentials names

De-fence-estration
No a fence
FenceBGone